In [1]:
! pip install transformers datasets
! pip install rouge-score nltk
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 15.7 MB/s 
     |████████████████████████████████| 441 kB 66.9 MB/s 
     |████████████████████████████████| 7.6 MB 51.3 MB/s 
     |████████████████████████████████| 163 kB 69.6 MB/s 
     |████████████████████████████████| 212 kB 73.8 MB/s 
     |████████████████████████████████| 115 kB 73.5 MB/s 
     |████████████████████████████████| 127 kB 75.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=4acf981e86e677401fb13ac6d05ef4da434c1464b45f02089a284cd00a4504d0
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3

In [2]:
!apt install git-lfs
#!git config --global user.email "rahulverma207788@gmail.com"
#!git config --global user.name "rahul77"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [3]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [4]:
import transformers
print(transformers.__version__)

4.23.1


In [5]:
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd
from datasets import load_dataset, dataset_dict, load_metric
from datasets.dataset_dict import DatasetDict
import datasets
import random
from IPython.display import display, HTML
from transformers import AutoTokenizer
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import requests

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
metric = load_metric("rouge")

### Loding the dataset

In [7]:
data = pd.read_csv("/content/the_hindu.csv")
data.drop(data.filter(regex="Unnamed"),axis=1, inplace=True)
data.head(3)

,Document,Original Summary,Corrected Summary
0,"Tripti Tandon, advocate for the Durbar Mahila ...","Supreme Court order on May 19 ""does not recogn...","The May 19 Supreme Court order ""does not recog..."
1,India's top oil and gas producer ONGC on Thurs...,India's top oil and gas producer ONGC says it ...,India's top oil and gas producer ONGC says it ...
2,Pakistan's National Assembly on May 26 passed ...,The Elections (Amendment) Bill 2022 passed wit...,The lower house of Pakistan's National Assembl...


In [8]:
data = data.drop(['Corrected Summary'], axis=1)

In [9]:
data['id'] = data[['Document', 'Original Summary']].sum(axis=1).map(hash)

In [10]:
data.head(3)

,Document,Original Summary,id
0,"Tripti Tandon, advocate for the Durbar Mahila ...","Supreme Court order on May 19 ""does not recogn...",-212597885655606684
1,India's top oil and gas producer ONGC on Thurs...,India's top oil and gas producer ONGC says it ...,-5905580444776036502
2,Pakistan's National Assembly on May 26 passed ...,The Elections (Amendment) Bill 2022 passed wit...,2826076963101056463


In [11]:
data.to_csv('the_hindu_1.csv')

In [12]:
dataset = load_dataset('csv', data_files='/content/the_hindu_1.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-fa56e4334c183cd9/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Document', 'Original Summary', 'id'],
        num_rows: 179
    })
})

In [14]:
dataset = dataset.remove_columns('Unnamed: 0')

### Splitting the dataset

In [15]:
train_devtest = dataset['train'].train_test_split(shuffle = True, seed = 200, test_size=0.3)
posts_dev_test = train_devtest['test'].train_test_split(shuffle = True, seed = 200, test_size=0.50)
raw_dataset = DatasetDict({
    'train': train_devtest['train'],
    'test': posts_dev_test['test'],
    'validation': posts_dev_test['train']})

In [16]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Document', 'Original Summary', 'id'],
        num_rows: 125
    })
    test: Dataset({
        features: ['Document', 'Original Summary', 'id'],
        num_rows: 27
    })
    validation: Dataset({
        features: ['Document', 'Original Summary', 'id'],
        num_rows: 27
    })
})

In [17]:
raw_dataset["train"][0]

{'Document': 'GUWAHATI The migration of border villagers has added to the challenges the Indian Army has been facing along the Line of Actual Control (LAC) in trying to match the rapid enhancement of infrastructure and operational capabilities by the People\'s Liberation Army (PLA) of China. Lieutenant General Rana Pratap Kalita, chief of the Army\'s Eastern Command, said this in Guwahati on Monday, while asserting that the partial withdrawal of the Armed Forces (Special Powers) Act from three northeastern States has aided the redeployment of troops on the northern borders of the eastern sector and focus more on China. "Migration of people from near the LAC in Arunachal Pradesh has led to shortage of people to work in those areas. The outmigration to the hinterland has been compounded by the unwillingness of people from outside to work in those difficult areas," he said. Hundreds of villagers have moved to towns in the more accessible areas in search of greener pastures over the years.

### Showing the elements

In [18]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [19]:
show_random_elements(raw_dataset["train"])

In [20]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [21]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

### Loding the tokenizer and Preprocessing the data

In [22]:
model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [23]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 19, 3, 9, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [24]:
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [25]:
max_input_length = 1024
max_target_length = 128
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["Original Summary"], max_length=max_target_length, truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [26]:
preprocess_function(raw_dataset["train"][:1])

{'input_ids': [[21603, 10, 3, 13961, 12054, 566, 21580, 37, 11661, 13, 4947, 3, 31101, 65, 974, 12, 8, 2428, 8, 2557, 6788, 65, 118, 5008, 590, 8, 4919, 13, 24193, 4330, 41, 4569, 254, 61, 16, 1119, 12, 1588, 8, 3607, 15220, 13, 3620, 11, 7763, 5644, 57, 8, 2449, 31, 7, 12774, 257, 6788, 41, 5329, 188, 61, 13, 1473, 5, 30943, 2146, 15039, 9, 6110, 8873, 5740, 155, 9, 6, 5752, 13, 8, 6788, 31, 7, 7345, 13901, 6, 243, 48, 16, 2846, 17771, 144, 23, 30, 2089, 6, 298, 15376, 53, 24, 8, 11807, 11905, 13, 8, 18251, 26, 5205, 7, 41, 7727, 23, 138, 2621, 7, 61, 1983, 45, 386, 25806, 49, 29, 1323, 65, 3, 5385, 26, 8, 1131, 15, 7379, 63, 297, 13, 12673, 30, 8, 8390, 15094, 13, 8, 13350, 2393, 11, 992, 72, 30, 1473, 5, 96, 329, 23, 122, 2661, 13, 151, 45, 1084, 8, 5292, 254, 16, 71, 4312, 9, 12654, 22660, 65, 2237, 12, 16316, 13, 151, 12, 161, 16, 273, 844, 5, 37, 91, 10673, 2661, 12, 8, 7841, 40, 232, 65, 118, 12771, 15, 26, 57, 8, 28963, 655, 13, 151, 45, 1067, 12, 161, 16, 273, 1256, 844, 976, 

In [27]:
tokenized_datasets = raw_dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### Loading the pretrained model

In [28]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


### Defining the parameters of the model

In [29]:
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 20
model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-thehindu"

In [30]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [31]:
tokenized_datasets["train"]

Dataset({
    features: ['Document', 'Original Summary', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 125
})

In [32]:
show_random_elements(tokenized_datasets["train"])

,Document,Original Summary,id,input_ids,attention_mask,labels
0,"The Indian auto component industry is expected to clock 8-10% growth in FY23, supported by the easing of supply-chain issues and commodity inflation in the second half of the year, according to rating agency Icra. For 2021-22, the revenue growth of the industry is pegged at 13-15%, driven by domestic OEM, replacement, export volumes and pass-through of commodity prices. The healthy volume growth will, however, come on a low base of FY2021, the agency said in a statement. ""Demand for auto components stems from domestic OEMs, replacement and exports. Domestic OEM demand has remained a mixed bag across segments in FY2022, with a slowdown in two-wheelers (2Ws) and semiconductor shortage dragging down overall production volumes,"" Icra Ltd. Assistant Vice President & Sector Head – Corporate Ratings Vinutaa S said. Exports have remained a bright spot in the Indian auto component story, partly aided by the ""China+1 strategy"". This is despite supply chain issues, she added. ""ICRA believes that the growth in FY2022 exports would have been even better if not for the semiconductor shortage. While auto ancillaries have a healthy export order book for the next few months, the impact of geopolitical and supply-chain issues on actual offtake remains monitorable,"" Ms. Vinutaa said. For FY2023, Icra said, ""Revenues are likely to expand by 8-10% supported by easing of supply-chain issues and commodity inflation in H2 FY2023. Over the long term, premiumisation of vehicles and focus on localisation will translate into healthy growth for auto component suppliers"". The rating agency said cost inflation and semiconductor shortage remain headwinds for the industry. ""Operating margins for auto ancillaries are likely to be impacted in the near term with elevated raw material, fuel and freight prices. While the semiconductor situation has been improving in the last 1-2 months, the Russia-Ukraine conflict could stress the globalised chip value chain,"" it said. In the aftermarket segment, the agency said the improvement in personal mobility, healthy freight movement and deferment of new vehicle purchases due to cost inflation have supported replacement sales in the last few months. Part of the revenue growth has also come from commodity pass-through. While January to mid-February was relatively dull because of the Omicron wave, demand has picked up in the last few weeks.","For 2021-22, the revenue growth of the industry is pegged at 13-15%, driven by domestic OEM, replacement, export volumes and pass-through of commodity prices . For FY2023, Icra said, ""Revenues are likely to expand by 8-10% supported by easing of supply-chain issues""",8584209773307906364,"[21603, 10, 37, 2557, 1510, 3876, 681, 19, 1644, 12, 6702, 505, 4536, 1454, 1170, 16, 23263, 2773, 6, 3510, 57, 8, 3, 16927, 13, 1899, 18, 19836, 807, 11, 23070, 12485, 16, 8, 511, 985, 13, 8, 215, 6, 1315, 12, 5773, 3193, 27, 2935, 5, 242, 460, 2658, 16149, 6, 8, 3751, 1170, 13, 8, 681, 19, 158, 11918, 44, 1179, 2292, 2712, 6, 6737, 57, 4422, 18430, 6, 3709, 6, 4202, 13548, 11, 1903, 18, 11258, 13, 23070, 1596, 5, 37, 1695, 2908, 1170, 56, 6, 983, 6, 369, 30, 3, 9, 731, 1247, 13, 23263, 19818, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]","[242, 460, 2658, 16149, 6, 8, 3751, 1170, 13, 8, 681, 19, 158, 11918, 44, 1179, 2292, 2712, 6, 6737, 57, 4422, 18430, 6, 3709, 6, 4202, 13548, 11, 1903, 18, 11258, 13, 23070, 1596, 3, 5, 242, 23263, 19818, 6355, 27, 2935, 243, 6, 96, 1649, 15098, 7, 33, 952, 12, 4405, 57, 505, 4536, 1454, 3510, 57, 3, 16927, 13, 1899, 18, 19836, 807, 121, 1]"
1,"Nepal is running low on foreign currency reserves needed to import medicines, oil pro

### Creating the dataset which is ready to pass model

In [33]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [34]:
from transformers import AdamWeightDecay
import tensorflow as tf
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [35]:
def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)
    return result

### Training the model

In [36]:
from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard
tensorboard_callback = TensorBoard(log_dir="./summarization_model_save/logs")
push_to_hub_callback = PushToHubCallback(
    output_dir="./summarization_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)
metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True
)
callbacks = [metric_callback, tensorboard_callback, push_to_hub_callback]
model.fit(
    train_dataset, validation_data=validation_dataset, epochs=20, callbacks=callbacks
)

Cloning https://huggingface.co/rahul77/t5-small-finetuned-thehindu into local empty directory.


Epoch 1/20
 6/15 [===========>..................] - ETA: 6s - loss: 0.9468

15/15 [==============================] - 85s 4s/step - loss: 0.8970 - val_loss: 0.6284 - rouge1: 32.7985 - rouge2: 24.3568 - rougeL: 30.6000 - rougeLsum: 32.0511 - gen_len: 19.0000
Epoch 2/20
15/15 [==============================] - 12s 806ms/step - loss: 0.7334 - val_loss: 0.5635 - rouge1: 33.5772 - rouge2: 26.4317 - rougeL: 31.2126 - rougeLsum: 32.5565 - gen_len: 19.0000
Epoch 3/20
15/15 [==============================] - 12s 803ms/step - loss: 0.6302 - val_loss: 0.5297 - rouge1: 32.7576 - rouge2: 25.9252 - rougeL: 30.6581 - rougeLsum: 31.8424 - gen_len: 19.0000
Epoch 4/20
15/15 [==============================] - 13s 869ms/step - loss: 0.5888 - val_loss: 0.5042 - rouge1: 31.4109 - rouge2: 23.3103 - rougeL: 29.1689 - rougeLsum: 30.2855 - gen_len: 19.0000
Epoch 5/20
15/15 [==============================] - 12s 823ms/step - loss: 0.5491 - val_loss: 0.4858 - rouge1: 30.1290 - rouge2: 22.3141 - rougeL: 28.2026 - rougeLsum: 29.0599 - gen_len: 19.0000
Epoch 6/20
15/15 [=====================

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.34k/231M [00:00<?, ?B/s]

Upload file logs/train/events.out.tfevents.1665957023.f9d0d1e421b7.68.0.v2:   0%|          | 3.34k/2.75M [00:0…

Upload file logs/validation/events.out.tfevents.1665957055.f9d0d1e421b7.68.1.v2: 100%|##########| 3.08k/3.08k …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/rahul77/t5-small-finetuned-thehindu
   8b60d8d..95fc924  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/rahul77/t5-small-finetuned-thehindu
   8b60d8d..95fc924  main -> main



### Loading the model and predicting the results

In [37]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
model_name = 'rahul77/t5-small-finetuned-thehindu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at rahul77/t5-small-finetuned-thehindu.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [38]:
def get_text_from_link(link):
    rsp = requests.get(link)
    if rsp.status_code != 200:
        return None
    soup = BeautifulSoup(rsp.text, features="lxml")
    all_p = soup.find_all("p")
    text = []
    for p in all_p:
        p_text = p.get_text()
        if (
            p.attrs != {} is not None
            or p_text.find("Photo Credit") != -1
            or p_text.find("Printable version") != -1
        ):
            continue
        p_text = (
            p_text.replace("‘", "'")
            .replace("’", "'")
            .replace("“", '"')
            .replace("”", '"')
            .replace(r"\'", "'")
        )
        text.append(p_text)
    text = " ".join(text)
    return text

In [39]:
document = get_text_from_link("https://www.thehindu.com/business/sebi-amends-rules-governing-collective-investment-schemes/article65271602.ece")

In [40]:
if 't5' in model_name: 
    document = "summarize: " + document
tokenized = tokenizer([document], return_tensors='np')
out = model.generate(**tokenized, max_length=128)

In [41]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0]))

<pad> Securities and Exchange Board of India (SEBI) approved an amendment to the SEBI (Collective Investment Schemes) Regulations, 1999 to strengthen the regulatory framework for collective investment schemes (CIS) in line with mutual fund regulations to remove regulatory arbitrage. Some of the key amendments included enhancement of net-worth criteria and requirement of having track record in relevant field as an eligibility requirement for registration as a collective investment management company (CIMC)</s>


### Using XLA and predicting results

In [42]:
import tensorflow as tf
@tf.function(jit_compile=True)
def generate(inputs):
    return model.generate(**inputs, max_length=128)
tokenized_data = tokenizer([document], return_tensors="np", pad_to_multiple_of=128)
out = generate(tokenized_data)

In [43]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0]))

<pad> Securities and Exchange Board of India (SEBI) approved an amendment to the SEBI (Collective Investment Schemes) Regulations, 1999 to strengthen the regulatory framework for collective investment schemes (CIS) in line with mutual fund regulations to remove regulatory arbitrage. Some of the key amendments included enhancement of net-worth criteria and requirement of having track record in relevant field as an eligibility requirement for registration as a collective investment management company (CIMC)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [44]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

Securities and Exchange Board of India (SEBI) approved an amendment to the SEBI (Collective Investment Schemes) Regulations, 1999 to strengthen the regulatory framework for collective investment schemes (CIS) in line with mutual fund regulations to remove regulatory arbitrage. Some of the key amendments included enhancement of net-worth criteria and requirement of having track record in relevant field as an eligibility requirement for registration as a collective investment management company (CIMC)


In [45]:
from transformers import pipeline
summarizer = pipeline('text2text-generation', model_name, framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at rahul77/t5-small-finetuned-thehindu.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [46]:
summarizer(document, max_length=128)

[{'generated_text': 'Securities and Exchange Board of India (SEBI) approved an amendment to the SEBI (Collective Investment Schemes) Regulations, 1999 to strengthen the regulatory framework for collective investment schemes (CIS) in line with mutual fund regulations to remove regulatory arbitrage . Some of the key amendments included enhancement of net-worth criteria and requirement of having track record in relevant field as an eligibility requirement for registration as a collective investment management company (CIMC)'}]